# Trouver l'elfe malveillant

Ce notebook t'accompagne pour enquêter dans la base SQLite **`elf_challenge.db`**.
Ton objectif : **identifier l'elfe malveillant**, démontrer la preuve avec **au moins 2 requêtes** et proposer **1 mesure corrective**.

**Tables** : `person`, `gift`, `transactions`, `access_log`.

---
**Conseil** : travaille par hypothèses, note tes trouvailles, et corrèle les logs d'accès avec les transactions autour d'horaires atypiques.


## 1) Préparation de l'environnement

- On va utiliser `sqlite3` et `pandas`.
- La cellule ci-dessous **vérifie** que la base `elf_challenge.db` existe.


In [ ]:
import os, sqlite3, random
from datetime import datetime, timedelta
import pandas as pd

DB_PATH = "elf_challenge.db"

def ensure_db(db_path: str):
    if os.path.exists(db_path):
        print("✔ Base SQLite trouvée :", db_path)
        return
    
    print("⚠ Base absente...", db_path)
   
ensure_db(DB_PATH)

## 2) Connexion & aperçu du schéma

In [ ]:
import sqlite3, pandas as pd
conn = sqlite3.connect(DB_PATH)

# Compter les lignes par table
q = (
    "SELECT 'person' as table_name, COUNT(*) as count FROM person "
    "UNION ALL SELECT 'gift', COUNT(*) FROM gift "
    "UNION ALL SELECT 'transactions', COUNT(*) FROM transactions "
    "UNION ALL SELECT 'access_log', COUNT(*) FROM access_log;"
)
pd.read_sql_query(q, conn)

In [ ]:
# Aperçu des colonnes de chaque table (pragma)
tables = ['person','gift','transactions','access_log']
for t in tables:
    df = pd.read_sql_query(f"PRAGMA table_info({t});", conn)
    print(f"\n-- {t} --")
    display(df)

## 3) Échauffement — requêtes simples

1. **Lister** les personnes avec leur rôle.  
2. **Compter** combien de cadeaux par statut (`status`).  
3. **Top** 10 transactions les plus récentes (avec nom de l'acteur).  


In [ ]:
# 1) Personnes / rôles
pd.read_sql_query("""
SELECT person_id, full_name, role
FROM person
ORDER BY role, full_name;
""", conn)

In [ ]:
# 2) Cadeaux par statut
pd.read_sql_query("""
SELECT status, COUNT(*) as nb
FROM gift
GROUP BY status
ORDER BY nb DESC;
""", conn)

In [ ]:
# 3) Top 10 transactions récentes
pd.read_sql_query("""
SELECT t.tx_id, t.gift_id, p.full_name AS actor, t.action, t.details, t.tx_ts
FROM transactions t
JOIN person p ON p.person_id = t.actor_id
ORDER BY t.tx_ts DESC
LIMIT 10;
""", conn)

## 4) Investigation — pistes guidées

### 4.1 Actions nocturnes (00:00–05:00)
- Qui agit la nuit ?
- Y a‑t‑il des **elfes** dans la liste ?


In [ ]:
pd.read_sql_query("""
SELECT p.person_id, p.full_name, a.ts, a.action
FROM access_log a
JOIN person p ON p.person_id = a.person_id
WHERE time(a.ts) BETWEEN '00:00:00' AND '05:00:00'
ORDER BY a.ts;
""", conn)

### 4.2 Transactions suspectes (emoji 🧝, 'missing')
- Chercher des **indices textuels**.


In [ ]:
pd.read_sql_query("""
SELECT t.tx_id, t.gift_id, t.actor_id, p.full_name, t.action, t.details, t.tx_ts
FROM transactions t
JOIN person p ON p.person_id = t.actor_id
WHERE t.details LIKE '%🧝%' OR t.details LIKE '%missing%'
ORDER BY t.tx_ts DESC;
""", conn)

### 4.3 IPs & devices partagés
- Plusieurs comptes depuis la **même IP / device** ?


In [ ]:
pd.read_sql_query("""
SELECT ip_addr, device, COUNT(DISTINCT person_id) as nb_accounts,
       GROUP_CONCAT(DISTINCT person_id) as person_ids
FROM access_log
GROUP BY ip_addr, device
HAVING nb_accounts > 1
ORDER BY nb_accounts DESC;
""", conn)

### 4.4 Corrélation `transactions` ↔ `access_log` (± 2 minutes)
- Valider qu'une action a eu lieu **pendant une session** donnée.


In [ ]:
pd.read_sql_query("""
SELECT t.tx_id, t.gift_id, t.actor_id, p.full_name as actor, t.details, t.tx_ts,
       a.log_id, a.person_id, a.ip_addr, a.device, a.location, a.action as access_action, a.ts as access_ts
FROM transactions t
JOIN person p ON p.person_id = t.actor_id
LEFT JOIN access_log a
  ON a.person_id = t.actor_id
 AND ABS(strftime('%s', t.tx_ts) - strftime('%s', a.ts)) <= 120
WHERE t.details LIKE '%missing%'
ORDER BY t.tx_ts DESC
LIMIT 50;
""", conn)

## 5) Ton enquête — espace de travail

Utilise les cellules ci‑dessous pour :  
- Formuler tes hypothèses
- Tester tes requêtes
- Consolider la **preuve** (au moins 2 requêtes pertinentes)


In [ ]:
# Preuve 1 — Toutes les transactions "missing 🧝" / "secret shelf 🧝" et leur auteur
preuve1 = pd.read_sql_query("""
SELECT
  t.tx_id, t.gift_id,
  p.person_id, p.full_name, p.role,
  t.action, t.details, t.tx_ts
FROM transactions t
JOIN person p ON p.person_id = t.actor_id
WHERE t.details LIKE '%🧝%'   -- marqueur très spécifique
   OR t.details LIKE '%secret shelf%'
ORDER BY t.tx_ts ASC;
""", conn)

preuve1

In [ ]:
# Preuve 2 — Corrélation transactions suspectes ↔ sessions d'accès (±2 minutes)
preuve2 = pd.read_sql_query("""
SELECT
  t.tx_id, t.gift_id,
  p.person_id, p.full_name, p.role,
  t.action AS tx_action, t.details AS tx_details, t.tx_ts,
  a.log_id, a.ts AS access_ts, a.action AS access_action,
  a.ip_addr, a.device, a.location, a.success
FROM transactions t
JOIN person p ON p.person_id = t.actor_id
LEFT JOIN access_log a
  ON a.person_id = t.actor_id
 AND ABS(strftime('%s', t.tx_ts) - strftime('%s', a.ts)) <= 120
WHERE (t.details LIKE '%🧝%' OR t.details LIKE '%secret shelf%')
ORDER BY t.tx_ts ASC;
""", conn)

preuve2

## 6) Conclusion — Rapport court

- **Elfe malveillant identifié** : `person_id = 15`, `full_name = Merry G. Night`
- **Preuve 1 (requête + interprétation)** :
    - Requête : filtre des transactions contenant **🧝** et/ou **“secret shelf”** (`preuve1`).
    - Interprétation : toutes les opérations “marked missing 🧝” et “moved to secret shelf 🧝” sont attribuées à **Merry G. Night (elf)**, souvent à des horaires **01:38–03:05** (nuit).
- **Preuve 2 (requête + interprétation)** :
    - Requête : jointure `transactions` ↔ `access_log` (± 2 minutes) (`preuve2`).
    - Interprétation : ces transactions correspondent à des sessions **edit_gift** réussies, localisées **warehouse**, depuis **10.0.0.45 / raspberry-pi-3**, au même moment : on a donc la **trace d’accès + l’action métier**, corrélées temporellement.
- **Mesure corrective (1)** :
    - Mettre en place une **règle de contrôle** : toute action `change_status -> missing` ou `transfer` vers une zone non standard (ex. “secret shelf”) nécessite une **double validation (4-eyes)** et/ou est **bloquée entre 00:00 et 05:00** sauf compte admin + justification (audit log obligatoire).